## Data

- Se obtuvo por un proceso de merge de 2 bases de datos el excel dfCuit el cual fue tratado manualmente para casos de coincidencias NO exactas.
- Se empleó la librería 'fuzzywuzzy' para calcular las probabilidades de que las razones sociales tanto como las direcciones dieran una coincidencia
- Se agregó una columna en la cual se identifica a las 'E-Commerce'
- Se agregó una columna en la que se identifica cuales direcciones no pertenecen a la ciudad. todas terminan con un paréntesis de cierre.
- Se agregó una columna en la cual se identifica el iltimo 'id_tramite'


In [ ]:
#pip install fuzzywuzzy

In [67]:
import pandas as pd
from laxFz import laxFz

url = r"../activa/data/dfCuit.xlsx"
df = pd.read_excel(url)


x = df["RAZON SOCIAL"].to_list()
y = df["razonsocial"].to_list()

arr = [laxFz(x[i],y[i]) for i in range(len(x))]

df["Pr(RAZONSOCIAL=razonsocial)"] = arr

x = df["DIRECCIÓN"].to_list()
y = (df["Calle"]+ " " + df["Numero"]).to_list()

arr = [laxFz(x[i],y[i]) for i in range(len(x))]

df["Pr(DIRECCIÓN=Calle + Num)"] = arr

df["E-Commerce"] = df["DIRECCIÓN"].map(lambda x: x=="E-Commerce")
df["No-Cba"] = df["DIRECCIÓN"].map(lambda x: x.strip()[-1]==")")


#identificacion de E-Commerce
def eCommerceUnique(row):
    if row['DIRECCIÓN'] == 'E-Commerce':
        return row['DIRECCIÓN'] + str(row['CUIT'])
    else:
        return row['DIRECCIÓN']

df['DIRECCIÓN'] = df.apply(eCommerceUnique, axis=1)


#add ast_id_Tramite


lastId_Tramite = {}

df["lastId_Tramite"] = 0

dfc = df[df["rt"] == 1]


for i in dfc["DIRECCIÓN"].unique():
    subset = dfc[dfc["DIRECCIÓN"] == i]
    if subset["id_tramite"].max():
        lastId_Tramite[i] = subset["id_tramite"].max()
    else:
        lastId_Tramite[i] = subset["id_tramite"][0]

for i in range(len(df)):
    r = df["DIRECCIÓN"][i]
    if r in lastId_Tramite.keys():
        df["lastId_Tramite"][i] = lastId_Tramite[r]

C:\Users\Usuario\AppData\Local\Temp\ipykernel_2964\4272864501.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lastId_Tramite"][i] = lastId_Tramite[r]


Excel para plot

In [69]:
dfk = df[df["rt"] == 1]
dfk = dfk.drop_duplicates(subset="ID").drop_duplicates(subset="DIRECCIÓN")
dfk.to_excel(r"../activa/data/ActivaToplot.xlsx")
len(dfk)


402

general excel

In [73]:
dfk = df.copy()
dfk = dfk.drop_duplicates(subset="ID").drop_duplicates(subset="DIRECCIÓN")
dfk.to_excel(r"../activa/data/ActivaGeneral.xlsx")
len(dfk)

504

In [71]:
len(lastId_Tramite)

402

Lista de chequeo primario de Activa (ignorar callen que empiecen con número "Ej: algunos casos de 9 de julio")

In [18]:

df[((df['UniqueId_x'] != df['UniqueId_y'])) & (df['rt'] == 1)].to_excel(r"../activa/data/CheckActiva.xlsx")

se trabajara para la descripcion con direcciones unicas correspondientes a la base de datos 'Activa'.

In [2]:

dfUniqueDir = df.copy()
dfUniqueDir["DIRECCIÓN"]

dfUniqueDir = df.drop_duplicates("DIRECCIÓN")

print(f'direcciones unicas: {len(dfUniqueDir)} ')
print(f'Fuera de Cba: {dfUniqueDir["No-Cba"].sum()}')
print(f'E-Commerce: {dfUniqueDir["E-Commerce"].sum()}')

direcciones unicas: 504 
Fuera de Cba: 29
E-Commerce: 4


Datos E-Commerce

In [3]:
df[df["E-Commerce"] == 1].to_excel(r"../activa/data/dataECommerce.xlsx")

Datos Fuera de Cba

In [4]:
df[df["No-Cba"] == 1].to_excel(r"../activa/data/dataNoCba.xlsx")

### datos para mapeo

en este caso de trabajara solo con los datos que en la verificacion tanto automatica como manual hayan obtenido 'True' en su verificacion, reasignado a la columna 'rt'. y su id_tramite sea UNICO.

In [39]:
dfx = df[df["rt"] == 1].copy()
dfx = dfx.drop_duplicates(subset=['id_tramite'])
dfx.to_excel(r"../activa/data/coincidencias-ciut.xlsx")
len(dfx)

583

si bien el id de tramite es unico, se estan dupicando direcciones, lo cual es impractico para el mapeo. se quitaran direccines distintas.

In [31]:
dfx = dfx.drop_duplicates(subset=['DIRECCIÓN'])
dfx.to_excel(r"../activa/data/coincidencias-ciut-direccion-unica.xlsx")

398
515


In [22]:
dr = pd.read_excel(r'C:\Users\Usuario\Desktop\Code\WorkSpace\activa\data\Activa_listado.xlsx',sheet_name=1)